# 

# Three products - Modelling

## Autores:
- Alejandro De Haro
- Daniel Escobosa
- Pablo Berastegui
- Jose María Benitez

In [2]:
# module for building the pyomo model
import pyomo.environ as pe
# module for solving the pyomo model
import pyomo.opt as po

In [3]:
model = pe.ConcreteModel()

Order to build the model:
1. Sets
1. Parameters
1. Variables
1. Objective function
1. Constraints

#### Sets

$p$: product {'A', 'B', 'C'}

$f$: factory {'F1', 'F2', 'F3', 'F4', 'F5', 'F6'}

In [4]:
model.product = pe.Set(initialize=['A', 'B', 'C'])
model.factory = pe.Set(initialize=[f'F{i}' for i in range(1, 7)])

#### Parameters

$C_{p,f}$: cost of product $p$ in factory $f$ [g/kg]

$NS_p$: number of product $p$ sold

$MC_f$: maximum capacity of factory $f$

$SP_p$: selling price of product $p$

In [5]:
cost = {
    ('A', 'F1'): 25,
    ('A', 'F2'): 30,
    ('A', 'F3'): 26,
    ('A', 'F4'): 34,
    ('A', 'F5'): 32,
    ('A', 'F6'): 30,

    ('B', 'F1'): 30,
    ('B', 'F2'): 32,
    ('B', 'F3'): 34,
    ('B', 'F4'): 35,
    ('B', 'F5'): 38,
    ('B', 'F6'): 40,

    ('C', 'F1'): 40,
    ('C', 'F2'): 46,
    ('C', 'F3'): 42,
    ('C', 'F4'): 37,
    ('C', 'F5'): 40,
    ('C', 'F6'): 50
}

n_sells = {
    'A': 700,
    'B': 500,
    'C': 600
}

maximum_capacity = {
    'F1': 550,
    'F2': 700,
    'F3': 1100,
    'F4': 350,
    'F5': 400,
    'F6': 450
}

selling_price = {
    'A': 60,
    'B': 82.5,
    'C': 108
}

model.cost = pe.Param(model.product, model.factory, initialize=cost)
model.n_sells = pe.Param(model.product, initialize=n_sells)
model.maximum_capacity = pe.Param(model.factory, initialize=maximum_capacity)
model.selling_price = pe.Param(model.product, initialize=selling_price)

#### Variables

$x_{p,f}$: quantity of product $p$ manufactured in factory $f$

In [6]:
model.x = pe.Var(model.product, model.factory, domain=pe.NonNegativeReals)

#### Objective function

Benefits = Earnings - Costs     (function to maximise)

- Earnings = $\sum_{p,f}$ $x_{p,f}$ $*$ $SP_p$
- Costs = $\sum_{p,f}$ $x_{p,f}$ $*$ $C_{p,f}$


**max $\sum_{p,f}$ $x_{p,f}$ $*$ $SP_p$ - $\sum_{p,f}$ $x_{p,f}$ $*$ $C_{p,f}$**

In [7]:
def obj_rule(model):
    return sum(model.x[p, f] * model.selling_price[p] for p in model.product for f in model.factory) - \
           sum(model.x[p, f] * model.cost[p, f] for p in model.product for f in model.factory)

model.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)

#### Constraints

$\sum_p$ $x_{p,f}$ ≤ $MC_f$ $∀ f$

$\sum_p$ $x_{p,f}$ ≥ $NS_p$ $∀ p$

$x_{p,f}$ ≥ 0 $∀ p,f$

In [8]:
model.max_production = pe.ConstraintList()

for f in model.factory:
    model.max_production.add(sum(model.x[p, f] for p in model.product) <= model.maximum_capacity[f])

model.min_sales = pe.ConstraintList()
for p in model.product:
    model.min_sales.add(sum(model.x[p, f] for f in model.factory) >= model.n_sells[p])

# The constraint of being non-negative is already included in the variable definition

#### Solver definition and solver statement

In [9]:
# falta instalar gurobi
solver = po.SolverFactory('gurobi')
results = solver.solve (model, tee=True)

Set parameter Username
Set parameter LicenseID to value 2704768
Academic license - for non-commercial use only - expires 2026-09-08
Read LP format model from file /var/folders/pr/nst7vndn0l10vqp0s73wclhr0000gn/T/tmpj8jbvlxa.pyomo.lp
Reading time = 0.00 seconds
x1: 9 rows, 18 columns, 36 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 18 columns and 36 nonzeros
Model fingerprint: 0x5bbc731c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6200000e+32   1.800000e+31   8.620000e+02      0s
      10    2.0520000e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 ite

In [ ]:
# Total benefit
print(round(pe.value(model.obj),2))

205200.0


In [ ]:
# Best production plan
for p in model.product:
    for f in model.factory:
        print('Amount of product {} in factory {} is: {}'.format(pe.value(p), pe.value(f),round(pe.value(model.x[p,f]),2)))

Amount of product A in factory F1 is: 0.0
Amount of product A in factory F2 is: 200.0
Amount of product A in factory F3 is: 50.0
Amount of product A in factory F4 is: 0.0
Amount of product A in factory F5 is: 0.0
Amount of product A in factory F6 is: 450.0
Amount of product B in factory F1 is: 0.0
Amount of product B in factory F2 is: 500.0
Amount of product B in factory F3 is: 0.0
Amount of product B in factory F4 is: 0.0
Amount of product B in factory F5 is: 0.0
Amount of product B in factory F6 is: 0.0
Amount of product C in factory F1 is: 550.0
Amount of product C in factory F2 is: 0.0
Amount of product C in factory F3 is: 1050.0
Amount of product C in factory F4 is: 350.0
Amount of product C in factory F5 is: 400.0
Amount of product C in factory F6 is: 0.0
